# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [1]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re
from collections import Counter
from nltk import ngrams
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import matplotlib as mpl
import spacy
from spacy import displacy
from newsapi import NewsApiClient
from path import Path
import csv
import warnings
warnings.filterwarnings('ignore')
nltk.download('vader_lexicon')
plt.style.use('seaborn-whitegrid')
mpl.rcParams['figure.figsize'] = [20.0, 10.0]
%matplotlib inline




Bad key savefig.frameon in file /Users/sameerlakhe/opt/anaconda3/envs/pyvizenv/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 421 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.4.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.level in file /Users/sameerlakhe/opt/anaconda3/envs/pyvizenv/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 472 ('verbose.level  : silent      # one of silent, helpful, debug, debug-annoying')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.4.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.fileo in file /Users/sameerlakhe/opt/anaconda3/envs/pyvizenv/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 473 ('verbose.fileo  : sys.stdout  #

In [2]:
# Read your api key environment variable
load_dotenv()
api_key = os.getenv("NEWS_API_KEY")

In [3]:
# Function to create a dataframe for english news and french news
def create_df(news, language):
    articles = []
    for article in news:
        try:
            title = article["title"]
            description = article["description"]
            text = article["content"]
            date = article["publishedAt"][:10]

            articles.append({
                "title": title,
                "description": description,
                "text": text,
                "date": date,
                "language": language
            })
        except AttributeError:
            pass

    return pd.DataFrame(articles)

#this function writes the event data to a csv file to avoid calling the API multiple times
def write_csv(df,file_name):
    file_path = Path(file_name)
    df.to_csv(file_path, index=False, encoding='utf-8-sig')


# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)
#get the news using News Api client
def get_news(query,language,file_name,reload=False):
    complete_file_name = "./Resources/"+file_name
    path = Path(complete_file_name) 
    #if reload is specified as true, then do not read from old file
    if (path.exists() and reload == False):
        print(f'The file {complete_file_name} exists')
        news_df = pd.read_csv(complete_file_name)
    else:
        print(f'retreiving data from new API for query: {query}, and language:{language}')
        results = newsapi.get_everything(
                    q=query,
                    language=language
                )
        print(f'total results for query "{query}" in langyage:{language} are : {results["totalResults"]}')
        #convert the news articles to dataframe
        news_df = create_df(results["articles"], language)
        write_csv(news_df,complete_file_name)
    return news_df
    

In [4]:
#reload is set to false, so it will retreive the data from already existing csv file
#pls specify last argument as True to reload fresh data
bitcoin_news_en_df = get_news("bitcoin","en","Bitcoin_en.csv",False)
bitcoin_news_en_df.head()


The file ./Resources/Bitcoin_en.csv exists


,title,description,text,date,language
0,Miami’s Bitcoin Conference Left a Trail of Har...,"For some women, inappropriate conduct from oth...","Now, even though there are a number of women-f...",2022-05-10,en
1,Why fossil fuel companies see green in Bitcoin...,ExxonMobil and other fossil fuel companies hav...,A Bitcoin mining site powered by otherwise los...,2022-05-04,en
2,Warren Buffett Says He Wouldn't Take All the B...,Warren Buffett has always been a bitcoin skept...,Warren Buffett has always been a bitcoin skept...,2022-05-02,en
3,Luna Foundation Tried to Prop Up Terra’s Crumb...,"As a kid, I remember when my father tried to u...","As a kid, I remember when my father tried to u...",2022-05-16,en
4,Bitcoin value drops by 50% since November peak,The slide in the value of cryptocurrencies com...,"Image source, Getty Images\r\nThe value of Bit...",2022-05-09,en


In [5]:
# Fetch the Ethereum news articles
#reload is set to false, so it will retreive the data from already existing csv file
#pls specify last argument as True to reload fresh data
ethereum_news_en_df = get_news("ethereum","en","Ethereum_en.csv",False)


The file ./Resources/Ethereum_en.csv exists


In [6]:
#dictionary to store the sentiment scores of the article
text_sent = {
        "text_compound": [],
        "text_pos": [],
        "text_neu": [],
        "text_neg": [],
        "text_sent": [],
        }

analyzer = SentimentIntensityAnalyzer()
# Get sentiment for the text and the title
def get_sentiment_score(news_df):
    
    for index, row in news_df.iterrows():
        try:
            # Sentiment scoring with VADER
            text_sentiment = analyzer.polarity_scores(row["text"])
            text_sent["text_compound"].append(text_sentiment["compound"])
            text_sent["text_pos"].append(text_sentiment["pos"])
            text_sent["text_neu"].append(text_sentiment["neu"])
            text_sent["text_neg"].append(text_sentiment["neg"])
            text_sent["text_sent"].append(get_sentiment(text_sentiment["compound"]))
        except AttributeError:
            pass
        
    return text_sent

    
# Attaching sentiment columns to the News DataFrame
bitcoin_text_sentiment_df = pd.DataFrame(text_sent)
bitcoin_news_en_df = bitcoin_news_en_df.join(bitcoin_text_sentiment_df)
bitcoin_news_en_df.head()

,title,description,text,date,language,text_compound,text_pos,text_neu,text_neg,text_sent
0,Miami’s Bitcoin Conference Left a Trail of Har...,"For some women, inappropriate conduct from oth...","Now, even though there are a number of women-f...",2022-05-10,en,NaN,NaN,NaN,NaN,NaN
1,Why fossil fuel companies see green in Bitcoin...,ExxonMobil and other fossil fuel companies hav...,A Bitcoin mining site powered by otherwise los...,2022-05-04,en,NaN,NaN,NaN,NaN,NaN
2,Warren Buffett Says He Wouldn't Take All the B...,Warren Buffett has always been a bitcoin skept...,Warren Buffett has always been a bitcoin skept...,2022-05-02,en,NaN,NaN,NaN,NaN,NaN
3,Luna Foundation Tried to Prop Up Terra’s Crumb...,"As a kid, I remember when my father tried to u...","As a kid, I remember when my father tried to u...",2022-05-16,en,NaN,NaN,NaN,NaN,NaN
4,Bitcoin value drops by 50% since November peak,The slide in the value of cryptocurrencies com...,"Image source, Getty Images\r\nThe value of Bit...",2022-05-09,en,NaN,NaN,NaN,NaN,NaN


In [7]:

# Attaching sentiment columns to the News DataFrame
ethereum_text_sentiment_df = pd.DataFrame(text_sent)
ethereum_news_en_df = ethereum_news_en_df.join(ethereum_text_sentiment_df)
ethereum_news_en_df.head()


,title,description,text,date,language,text_compound,text_pos,text_neu,text_neg,text_sent
0,Here's what NFTs look like on Instagram,Meta has revealed more of how NFTs will work o...,Meta has revealed more of how NFTs will work o...,2022-05-10,en,NaN,NaN,NaN,NaN,NaN
1,Yuga Labs apologises after sale of virtual lan...,Demand for plots of land in multiplayer game O...,A multi-billion dollar cryptocurrency company ...,2022-05-02,en,NaN,NaN,NaN,NaN,NaN
2,How Bored Ape Yacht Club Broke Ethereum - CNET,"Transactions stalled for hours, and people pai...",When Bored Ape Yacht Club creators Yuga Labs a...,2022-05-04,en,NaN,NaN,NaN,NaN,NaN
3,Cryptoverse: Ether prepares for epic 'merge' i...,Ether has promised to do better. It has promis...,April 26 (Reuters) - Ether has promised to do ...,2022-04-26,en,NaN,NaN,NaN,NaN,NaN
4,What is the Ethereum merge and how could it ma...,Ethereum is preparing for an upgrade that’s be...,Ethereum is preparing for an upgrade thats bee...,2022-04-26,en,NaN,NaN,NaN,NaN,NaN


In [8]:
bitcoin_text_sentiment_df.describe()

,text_compound,text_pos,text_neu,text_neg,text_sent
count,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN


In [9]:
ethereum_text_sentiment_df.describe()

,text_compound,text_pos,text_neu,text_neg,text_sent
count,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN


### Questions:

Q: Which coin had the highest mean positive score?

A: ***Both coins have same mean positive score of 0.054675***

Q: Which coin had the highest compound score?

A: ***Ethereum has the highest compound score of 0.865500***

Q. Which coin had the highest positive score?

A: ***Ethereum has the highest positive score of 0.269000***


---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [10]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()

# Create a list of stopwords
sw = set(stopwords.words('english'))

In [11]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    # Remove the punctuation from text
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('', text)
    # Create a tokenized list of the words
    words = word_tokenize(re_clean)
    # Lemmatize words into root words
    lem = [lemmatizer.lemmatize(word) for word in words]
    # Convert the words to lowercase
    # Remove the stop words
    tokens = [word.lower() for word in lem if word.lower() not in sw]
    return tokens


In [12]:
# Create a new tokens column for Bitcoin
def generate_tokens(df):
    list_tokens=[]
    for row in df["text"]:
        tokens = tokenizer(row)
        list_tokens.append(tokens)
    return list_tokens

bitcoin_tokens = generate_tokens(bitcoin_news_en_df)
bitcoin_news_en_df['tokens'] = bitcoin_tokens
bitcoin_news_en_df.head()

,title,description,text,date,language,text_compound,text_pos,text_neu,text_neg,text_sent,tokens
0,Miami’s Bitcoin Conference Left a Trail of Har...,"For some women, inappropriate conduct from oth...","Now, even though there are a number of women-f...",2022-05-10,en,NaN,NaN,NaN,NaN,NaN,"[even, though, number, womenfocused, crypto, s..."
1,Why fossil fuel companies see green in Bitcoin...,ExxonMobil and other fossil fuel companies hav...,A Bitcoin mining site powered by otherwise los...,2022-05-04,en,NaN,NaN,NaN,NaN,NaN,"[bitcoin, mining, site, powered, otherwise, lo..."
2,Warren Buffett Says He Wouldn't Take All the B...,Warren Buffett has always been a bitcoin skept...,Warren Buffett has always been a bitcoin skept...,2022-05-02,en,NaN,NaN,NaN,NaN,NaN,"[warren, buffett, ha, always, bitcoin, skeptic..."
3,Luna Foundation Tried to Prop Up Terra’s Crumb...,"As a kid, I remember when my father tried to u...","As a kid, I remember when my father tried to u...",2022-05-16,en,NaN,NaN,NaN,NaN,NaN,"[kid, remember, father, tried, use, broom, han..."
4,Bitcoin value drops by 50% since November peak,The slide in the value of cryptocurrencies com...,"Image source, Getty Images\r\nThe value of Bit...",2022-05-09,en,NaN,NaN,NaN,NaN,NaN,"[image, source, getty, imagesthe, value, bitco..."


In [13]:
# Create a new tokens column for Ethereum
ethereum_tokens = generate_tokens(ethereum_news_en_df)
ethereum_news_en_df['tokens'] = ethereum_tokens
ethereum_news_en_df.head()

,title,description,text,date,language,text_compound,text_pos,text_neu,text_neg,text_sent,tokens
0,Here's what NFTs look like on Instagram,Meta has revealed more of how NFTs will work o...,Meta has revealed more of how NFTs will work o...,2022-05-10,en,NaN,NaN,NaN,NaN,NaN,"[meta, ha, revealed, nfts, work, instagram, us..."
1,Yuga Labs apologises after sale of virtual lan...,Demand for plots of land in multiplayer game O...,A multi-billion dollar cryptocurrency company ...,2022-05-02,en,NaN,NaN,NaN,NaN,NaN,"[multibillion, dollar, cryptocurrency, company..."
2,How Bored Ape Yacht Club Broke Ethereum - CNET,"Transactions stalled for hours, and people pai...",When Bored Ape Yacht Club creators Yuga Labs a...,2022-05-04,en,NaN,NaN,NaN,NaN,NaN,"[bored, ape, yacht, club, creator, yuga, labs,..."
3,Cryptoverse: Ether prepares for epic 'merge' i...,Ether has promised to do better. It has promis...,April 26 (Reuters) - Ether has promised to do ...,2022-04-26,en,NaN,NaN,NaN,NaN,NaN,"[april, reuters, ether, ha, promised, better, ..."
4,What is the Ethereum merge and how could it ma...,Ethereum is preparing for an upgrade that’s be...,Ethereum is preparing for an upgrade thats bee...,2022-04-26,en,NaN,NaN,NaN,NaN,NaN,"[ethereum, preparing, upgrade, thats, describe..."


---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [14]:
#concat data from all the rows for tokens column
def get_big_token_string(df):
    big_str=''
    for row in df["tokens"]:
        list_string = ' '.join(row)
        big_str = big_str + list_string
    return big_str



In [15]:
# Generate the Bitcoin N-grams where N=2
big_string_bitcoin_tokens = get_big_token_string(bitcoin_news_en_df)
bitcoin_tokens = big_string_bitcoin_tokens.split()
bitcoin_bigram_counts = Counter(ngrams(bitcoin_tokens, n=2))

In [16]:
# Generate the Ethereum N-grams where N=2
big_string_ethereum_tokens = get_big_token_string(ethereum_news_en_df)
ethereum_tokens = big_string_ethereum_tokens.split()
ethereum_bigram_counts = Counter(ngrams(ethereum_tokens, n=2))

In [17]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [18]:
# Use token_count to get the top 10 words for Bitcoin
bitcoin_top_words = token_count(bitcoin_tokens,10)
print(bitcoin_top_words)

[('bitcoin', 13), ('cryptocurrency', 11), ('world', 8), ('ha', 7), ('reuters', 7), ('week', 5), ('investor', 4), ('dropped', 4), ('charmay', 4), ('wednesday', 4)]


In [19]:
# Use token_count to get the top 10 words for Ethereum
ethereum_top_words = token_count(ethereum_tokens,10)
print(ethereum_top_words)

[('cryptocurrency', 9), ('bitcoin', 8), ('ha', 7), ('world', 7), ('biggest', 5), ('reuters', 5), ('last', 5), ('ethereum', 4), ('nft', 4), ('week', 4)]


---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [ ]:
# Generate the Bitcoin word cloud
wc_bitcoin = WordCloud(width=1800, height=1000, max_words=50).generate(big_string_bitcoin_tokens)
plt.imshow(wc_bitcoin)



In [ ]:
# Generate the Ethereum word cloud
wc_ethereum = WordCloud(width=1800, height=1000, max_words=50).generate(big_string_ethereum_tokens)
plt.imshow(wc_ethereum)

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [ ]:
# Download the language model for SpaCy
!python -m spacy download en_core_web_sm

In [ ]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [ ]:
# Concatenate all of the Bitcoin text together
def concat_news_data(df):
    big_str=''
    for row in df["text"]:
        list_string = ''.join(row)
        big_str = big_str + list_string
    return big_str

In [ ]:
# Run the NER processor on all of the text
# Set article to be analyzed with spaCy
bitcoin_en_news_data = concat_news_data(bitcoin_news_en_df)
doc = nlp(bitcoin_en_news_data)
# Add a title to the document
doc.user_data["title"] = "Bitcoin Named Entity Recognition(NER)"

In [ ]:
# Render NER visualization with displacy
displacy.render(doc, style='ent')

In [ ]:
# List all Entities
print([ent.text for ent in doc.ents])

In [ ]:
displacy.render(doc, style='dep')

---

### Ethereum NER

In [ ]:
# Concatenate all of the Ethereum text together
ethereum_all_en_news_data = concat_news_data(ethereum_news_en_df)


In [ ]:
# Run the NER processor on all of the text
doc = nlp(ethereum_all_en_news_data)

# Add a title to the document
doc.user_data["title"] = "Ethereum Named Entity Recognition(NER)"

In [ ]:
# Render the visualization
displacy.render(doc, style='ent')

In [ ]:
# List all Entities
print([ent.text for ent in doc.ents])

In [ ]:
displacy.render(doc, style='dep')

---